In [1]:

%load_ext autoreload
%autoreload 2
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import numpy as np
import torch

from transformers import pipeline

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import GPT2Config, GPT2Model,PreTrainedModel,PretrainedConfig
from MoEWrapper import MoEWrapper
cfg=PretrainedConfig()
cfg=GPT2Config()

moe=MoEWrapper(cfg)

model_name_list =[ 
    "../models/hf/0401_270m_eng",
    "../models/hf/0401_270m_web",
    "../models/hf/0401_270m_fin",
                  ]

tokenizer = AutoTokenizer.from_pretrained(model_name_list[0])
for model_name in model_name_list:
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map="auto",
        torch_dtype=torch.float16
    )
    moe.append_model(model)

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
max_new_tokens=30
repetition_penalty=2.0
pipe=pipeline("text-generation",model=moe,
              tokenizer=tokenizer,
     max_new_tokens=max_new_tokens,
                      repetition_penalty=repetition_penalty,
 pad_token_id=50256,
              )



The model 'MoEWrapper' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM

In [7]:
inputs=[
    "hello, I'm John",
    "global warming ",
    "英語: He is a good man. 日本語: ",
    "今日は晴れてるから",
    "ブログを書きました",
    "地球温暖化を防ぐに",
    "ガリレオによるピサの斜塔実験とは",
    "スーパーコンピューターの富岳は",
]
moe.set_flexible_mode()
for text in inputs:
    print("------")
    print("input: ", text)
    print(pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model 2 will be used
ppl array:  [283.75 325.5  192.  ]
hello, I'm John Doe."  1973: "They are the ones who have been in charge of this country for so long and
------
input:  global warming 
model 0 will be used
ppl array:  [ 725.  4404.   847.5]
global warming 10–25 °C, and then to a temperature of about-38.4° C in August; it was reported that
------
input:  英語: He is a good man. 日本語: 
model 2 will be used
ppl array:  [93.25  69.    45.625]
英語: He is a good man. 日本語: 彼は素晴らしい人です。  「He's the one who made me feel like I was in love with him/But he doesn’t
------
input:  今日は晴れてるから
model 0 will be used
ppl array:  [ 239.   956.5 6772. ]
今日は晴れてるから、雨が降らないと困る。 でもね～・・・この前は雨が降ってなかったのにな～。 【楽天市場】10%OFF
------
input:  ブログを書きました
model 0 will be used
ppl array:  [ 314.25  446.5  2064.  ]
ブログを書きました。 2017/3月5日(水)の朝、東京・新宿で「第6回『日本と世界の平和
------
input:  地球温暖化を防ぐに
model 0 will be used
ppl array:  [ 136.25 1167.   2844.  ]
地球温暖化を防ぐにはどうしたらいいのでしょうか。 「宇宙は、人類が作り出したものだ」という考えを広める活動を

In [8]:
for text in inputs:
    print("------")
    print("input: ", text)
    for model_id in range(len(model_name_list)):
        moe.fix_model(model_id)
        print(f"model id {model_id}: ",pipe(text)[0]["generated_text"].replace("\n"," "))

------
input:  hello, I'm John
model id 0:  hello, I'm John.  1970s: The Last of the Seven Stars (The Four Star) - A short film about a group called "the
model id 1:  hello, I'm John and then a little bit of my life."  2018: "They are not really in love with me but have
model id 2:  hello, I'm John Doe."  1973: "They are the ones who have been in charge of this country for so long and
------
input:  global warming 
model id 0:  global warming 10–25 °C, and then to a temperature of about-38.4° C in August; it was reported that
model id 1:  global warming 2018年7月3日 【動画】中国で「日本製品不買運動」が活発化、日本の食品メーカー
model id 2:  global warming climatic conditions. The climate change scenarios are based on the assumption that a large part of Earth'need is being lost due to human
------
input:  英語: He is a good man. 日本語: 
model id 0:  英語: He is a good man. 日本語: 'I am not the best of my friends, but I have always been in love with you forever and it will be better than that which would
model id 1:  英語: He is a